In [ ]:
using GeoDataFrames
using DataFrames
using Plots
using Shapefile
using Random
using ArchGDAL
using GeoInterface

In [ ]:
###Shapefile municipalities

shapefile_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = Shapefile.Table(shapefile_path) |> DataFrame

# Highway shapefile
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = Shapefile.Table(shapefile_path_highways) |> DataFrame

In [ ]:
# Filtering sg_uf equals to MT
filtered_highways_MT = filter(row -> row.sg_uf == "MT", highways)

# Grouping by vl_br row
grouped_highways = groupby(filtered_highways_MT, :vl_br)

In [ ]:
#Combine geometries
function combine_geometries(geometries)

    return mapfoldl(
        x -> GeoInterface.convert(ArchGDAL, x), # map - apply to each element
        ArchGDAL.union, # reduce 2 elements to 1 element
        geometries # the vector to perform the operation on
    )
end

In [ ]:
# Applying function
combined_geometries = combine(grouped_highways, :geometry => combine_geometries => :geometry)

#Showing results
println(combined_geometries)

In [ ]:
# Gerar cores aleatórias para cada geometria
colors = [RGB(rand(), rand(), rand()) for _ in 1:length(combined_geometries.geometry)]

In [ ]:
# Plotar todos os municípios e as rodovias filtradas
plot(size=(1000, 800))  # Definir tamanho do gráfico
plot!(legend=:outerright)
plot!(municipalities.geometry, color=:white, linecolor=:black, label=false)

In [ ]:
# Plotar as rodovias com cores diferentes e adicionar legenda
for (i, geom) in enumerate(combined_geometries.geometry)
    plot!(geom, color=colors[i], linewidth=4, label="BR-" * string(combined_geometries.vl_br[i]))
end

In [ ]:
# Ajustar a legenda
plot!(legend=:outerright, legendfontsize=8)

# Remover eixos e grades
plot!(framestyle=:none, grid=false)

# Ajustar títulos e rótulos dos eixos
xlabel!("")
ylabel!("")
title!("")

In [ ]:
# Salvar o gráfico
savefig("C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Graph-Rodovias.png")